In [1]:
import xarray as xr
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import gsw

In [2]:
sd = 1 # Sill depth
file1 = xr.open_dataset('/Users/weiyangbao/Documents/Model_outputs/NoTide/state_' + str(format(sd,'03d')) + '.nc')
state1 = file1.isel(X=range(120), Xp1=range(121), Y=range(30,39), Yp1=range(30,40)) # Remove the boundary walls and the shelf region
file2 = xr.open_dataset('/Users/weiyangbao/Documents/Model_outputs/qf-tide/state_' + str(format(sd,'03d')) + '.nc')
state2 = file2.isel(X=range(120), Xp1=range(121), Y=range(30,39), Yp1=range(30,40)) # Remove the boundary walls and the shelf region

time = state1.T.data    
x = state1.X.data / 1000 # x coornidate in km
z = state1.Z.data
pres = gsw.p_from_z(z, 0)
p = np.tile(pres,[len(x),1]).T


# NoTide data
pt1 = state1.Temp.data.mean(2) # Dimesions T Z X
s1 = state1.S.data.mean(2)
rho1 = gsw.rho(s1, pt1, p)
u1_0 = 0.5 * (state1.U.data[:, :, :, 1:] + state1.U.data[:, :, :, :-1])
u1 = u1_0.mean(2)
w1 = state1.W.data.mean(2)
mask1 = (pt1 == 0)
pt1[mask1] = np.nan
s1[mask1] = np.nan
u1[mask1] = np.nan
w1[mask1] = np.nan
rho1[mask1] = np.nan


# qf-tide data
pt2 = state2.Temp.data.mean(2) # Dimesions T Z X
s2 = state2.S.data.mean(2)
rho2 = gsw.rho(s2, pt2, p)
u2_0 = 0.5 * (state2.U.data[:, :, :, 1:] + state2.U.data[:, :, :, :-1])
u2 = u2_0.mean(2)
w2 = state2.W.data.mean(2)
mask2 = (pt2 == 0)
pt2[mask2] = np.nan
s2[mask2] = np.nan
u2[mask2] = np.nan
w2[mask2] = np.nan
rho2[mask2] = np.nan

In [3]:
fig_path = "/Users/weiyangbao/Documents/Results_Figs/Af_compare/Sill009/"
if not os.path.exists(fig_path):
    os.makedirs(fig_path)


    
ct = np.linspace(6,9,10) # Contour range for temperature
crho = np.linspace(1016,1020,10)
cmap = 'RdYlBu_r'

for t in range(120,169):   
    fig = plt.figure(figsize=(16,8))
    fig.tight_layout()

    # NoTide Plot
    ax1 = fig.add_subplot(211)
    im1 = ax1.contourf(x,z[:24],pt1[t,:24,:], ct, cmap=cmap)
    ax1.xaxis.set_visible(False)
    ax1.set_xlim([x[1],x[-1]])
    ax1.set_facecolor('gray')

    cbaxes = fig.add_axes([0.74, 0.92, 0.15, 0.01])
    cbar = fig.colorbar(im1,cax=cbaxes, orientation='horizontal')
    cbar.set_ticks([6, 7, 8, 9])
    cbar.ax.set_title('Potential Temperature (\N{DEGREE SIGN}C)', size=10, weight='bold')
    # Plot isopycnals
    CS1 = ax1.contour(x,z[:24],rho1[t,:24,:],crho, colors = 'b')
    plt.clabel(CS1, fmt='%1.1f', fontsize=15, inline=1)
    ax1.set_ylabel('Depth (m)', size=15)
    ax1.tick_params(labelsize=12)
    ax1.set_title('Qf-Only', loc='left', size=15, weight='bold')
    ax1.set_title('Hour ' + str(t), loc='center', size=20, weight='bold')
    
    ut1 = u1[t,:,:]
    wt1 = w1[t,:,:]
    ut1[[1,2,4,5,7,8,10,12,14,16,18,20,22,24,26,28], :] = np.nan
    wt1[[1,2,4,5,7,8,10,12,14,16,18,20,22,24,26,28], :] = np.nan
    ut1[::2,:] = np.nan
    wt1[::2,:] = np.nan   
    Q = ax1.quiver(x[::5],z[:24],0.1*ut1[:24,::5], 0.1*wt1[:24,::5], width=0.001, scale=1)
    ax1.quiverkey(Q, 0.7, 0.89, 0.02,r'0.2 $m/s$', fontproperties={'weight': 'bold', 'size' : 10}, coordinates='figure')



    # qf-tide Plot
    ax2 = fig.add_subplot(212)
    im2 = ax2.contourf(x,z[:24],pt2[t,:24,:], ct, cmap=cmap)
    ax2.set_xlim([x[1],x[-1]])
    ax2.set_facecolor('gray')
    CS2 = ax2.contour(x,z[:24],rho2[t,:24,:],crho, colors = 'b')
    plt.clabel(CS2, fmt='%1.1f', fontsize=15, inline=1)
    ax2.set_ylabel('Depth (m)', size=15)
    ax2.tick_params(labelsize=12)
    ax2.set_title('Qf-Tide', loc='left', size=15, weight='bold')
    
    ut2 = u2[t,:,:]
    wt2 = w2[t,:,:]
    ut2[[1,2,4,5,7,8,10,12,14,16,18,20,22,24,26,28], :] = np.nan
    wt2[[1,2,4,5,7,8,10,12,14,16,18,20,22,24,26,28], :] = np.nan
    ut2[::2,:] = np.nan
    wt2[::2,:] = np.nan   
    ax2.quiver(x[::5],z[:24],0.1*ut2[:24,::5], 0.1*wt2[:24,::5], width=0.001, scale=1)
    ax2.set_xlabel('Distance from glacier (km)', size=15)
    ax2.tick_params(labelsize=12)
    plt.close()
    fig.savefig(fig_path + '/time_r' + format(t, '03d') + '.png', dpi = 150, bbox_inches='tight')

In [4]:
# Generate GIF from saved output figures
import imageio
import glob

images = []
imagenames = sorted(glob.glob(fig_path + '/time_r*.png'))

for filename in imagenames:
    images.append(imageio.imread(filename))
imageio.mimsave(fig_path + 'Comp009_r.gif', images, duration=.5)